# Hands On - Aprendizado Federado aplicado à Internet das Coisas

**Notebook 2**: Criação de clientes no ambiente federado

O reconhecimento da atividade humana é uma área de pesquisa ativa e que possui um enorme potencial de benefício com o uso de aprendizado federado (FL), já que tais dados são normalmente privados e possuem informações sensíveis sobre os usuários.
Além disso, com FL também podemos desenvolver um modelo conjunto que consiga capturar a diversidade dos dados, algo que é extremamente difícil de ser coletado de forma individual.

Sob esse contexto, nesse tutorial vamos aprender como definir clientes para o treinamento federado de uma rede neural para auxilar no reconhecimento de atividades humanas (*Human Activity Recognition* - HAR) usando o framework de aprendizado federado
Flower em conjunto com a biblioteca de deep learning Pytorch.

### Dataset

Os dados serão particionados horizontalmente, assim os subconjuntos de treinamento e teste irão ser divididos em mini-batches (pequenos lotes) com base no número total de clientes.

Para isso, aplicaremos uma função auxiliar para carregar os dados e definir os conjuntos de treinamento e teste.
Nessa função, precisaremos dos seguintes parâmetros: 

* **data root (str)**: Diretório onde os datasets finais serão armazenados. 

* **train batch size (int)**: Tamanho do mini-batch usado nos dados de treinamento.

* **test batch size (int)**: Tamanho do mini-batch usado nos dados de teste. 

* **id (int)**: Client ID usado para selecionar uma partição específica. 

* **nb clients (int)**: Número total de clientes usados no treinamento.

In [1]:
#Carregando os dados
import flwr as fl
import torch
import aux

DATA_ROOT = "./data/pml-training.csv"

server_address = "[::]:8081"
cid = 1
nb_clients = 2
train_batch_size = 64
test_batch_size = 64
epochs = 10

# Load data
train_loader, test_loader = aux.load_data(
        data_root = DATA_ROOT,
        train_batch_size = train_batch_size,
        test_batch_size = test_batch_size,
        cid = cid,
        nb_clients = nb_clients + 1,
)

### Rede Neural

Atualmente o modelo de classificação mais adequado e vantajoso para a modelagem de um ambiente federado são as redes neurais.
Definimos essa configuração de arquitetura por meio da criação de uma classe em Pytorch denominada **HARmodel** presente no arquivo auxiliar *aux.py* adicionado.

### Cliente Flower

O próximo passo é definir a alocação dos dispositivos no ambiente federado. 

Quando o servidor seleciona um dispositivo específico do ambiente federado para realizar um treinamento, ele envia as instruções pela rede, por meio de uma interface chamada **Client**.
Assim, o cliente recebe as instruções do servidor e chama um dos métodos desta classe para executar seu código (ou seja, para treinar a sua rede neural local). 

O framework Flower fornece uma classe chamada *NumPyClient*, que torna mais fácil implementar a interface do cliente quando utilizamos PyTorch. 
Quando implementamos um NumPyClient devemos definir os seguintes métodos: 

* **get_parameters**: retorna o peso do modelo
como uma lista de ndarrays 

* **set_parameters** (opcional): atualiza os pesos do modelo
local com os parâmetros recebidos do servidor 

* **fit**: define os pesos do modelo local, treina o modelo localmente e recebe o update dos pesos locais 

* **evaluate**: define como o modelo local será testado. 

Abaixo mostramos como a classe Client foi implementada
para o caso de estudo apresentado:

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate client
client = aux.FlowerClient(
    cid = cid,
    train_loader = train_loader,
    test_loader = test_loader,
    epochs = epochs,
    device = device,
)

### Inicializando o cliente

O flower nos fornece a possibilidade de rodar o servidor e o cliente na mesma máquina, configurando o endereço do servidor como "[::]: 8080". 
Porém, se quisermos implementar uma aplicação realmente federada com o servidor e clientes em execução em diferentes máquinas, precisaremos apenas alterar o server address para o respectivo endereço da máquina do cliente.

In [3]:
# Código de instanciação do cliente 
fl.client.start_client(server_address, client)

DEBUG flower 2021-08-19 22:31:09,619 | connection.py:36 | ChannelConnectivity.IDLE
DEBUG flower 2021-08-19 22:31:09,621 | connection.py:36 | ChannelConnectivity.READY
INFO flower 2021-08-19 22:31:09,622 | app.py:61 | Opened (insecure) gRPC connection


Training 10 epoch(s) w/ 103 mini-batches each

Train Epoch: 0 [6336/6592 (96%)] Loss: 1.596585, Acc: 0.221749 (Cliente 1)				
Train Epoch: 1 [6336/6592 (96%)] Loss: 1.424146, Acc: 0.291193 (Cliente 1)				
Train Epoch: 2 [6336/6592 (96%)] Loss: 1.340835, Acc: 0.347538 (Cliente 1)				
Train Epoch: 3 [6336/6592 (96%)] Loss: 1.329292, Acc: 0.389047 (Cliente 1)				
Train Epoch: 4 [6336/6592 (96%)] Loss: 1.259240, Acc: 0.431976 (Cliente 1)				
Train Epoch: 5 [6336/6592 (96%)] Loss: 1.272115, Acc: 0.458333 (Cliente 1)				
Train Epoch: 6 [6336/6592 (96%)] Loss: 1.290238, Acc: 0.487689 (Cliente 1)				
Train Epoch: 7 [6336/6592 (96%)] Loss: 1.295281, Acc: 0.509785 (Cliente 1)				
Train Epoch: 8 [6336/6592 (96%)] Loss: 1.069018, Acc: 0.523201 (Cliente 1)				
Training 10 epoch(s) w/ 103 mini-batches each50, Acc: 0.536143 (Cliente 1)				

Train Epoch: 0 [6336/6592 (96%)] Loss: 1.174034, Acc: 0.508049 (Cliente 1)				
Train Epoch: 1 [6336/6592 (96%)] Loss: 1.116313, Acc: 0.527778 (Cliente 1)				
Trai

DEBUG flower 2021-08-19 22:32:15,453 | connection.py:68 | Insecure gRPC channel closed
INFO flower 2021-08-19 22:32:15,471 | app.py:72 | Disconnect and shut down


Client 1 - Evaluate on 6540 samples: Average loss: 0.0083, Accuracy: 81.27%

